<a href="https://colab.research.google.com/github/VerebicsPeter/CodeSim/blob/main/model/model_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Similarity with Fine-tuning

## Dependencies

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint as pp
import random
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset, random_split
# Hugging Face Transformers (fro CodeBERT etc.)
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
# Libraries for logging
from tqdm.auto import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

Device: cuda
Tesla T4


## Seeding

In [5]:
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)

set_seed(seed_value=42)

## Dataset Access

In [6]:
from google.colab import userdata

paired_dataset_url = f"https://drive.google.com/uc?export=download&id={userdata.get('codenetSamplePaired')}"
#paired_dataset_url = "https://drive.google.com/uc?export=download&id=1I3mbw4CIMij44vUrmf9XULZyg5lXHKwn"  # old dataset
#paired_dataset_url = "https://drive.google.com/uc?export=download&id=1Mdp7FX4YBgv3i0ga69e8gxcbDvSIngDM"  # new dataset

## Dataset

In [7]:
COLUMNS = [
    'pid',    # CodeNet problem ID
    'sid_1',  # CodeNet solution ID of 'src_1'
    'sid_2',  # CodeNet solution ID of 'src_2'
    'src_1',  # CodeNet solution code of 'sid_1'
    'src_2',  # CodeNet solution code of 'sid_2'
    'label',  # Label indicating if 'src_1' and 'src_2' both solve 'pid'
]

In [8]:
df = pd.read_csv(paired_dataset_url, header=0, names=COLUMNS)
df = df.drop(columns=['pid', 'sid_1', 'sid_2'])
df.head()

,src_1,src_2,label
0,"k, x = map(int, input().split())\nif x <= 500*...","N, X = map(int, input().split())\nprint(""Yes"")...",1
1,import sys\ninput=sys.stdin.readline\nsys.setr...,"from collections import defaultdict, deque, Co...",1
2,"x = int(input())\nif x < 1200:\n print(""ABC...","if int(input()) < 1200:\n print(""ABC"")\nelse:...",1
3,"H,W = map(int,input().split())\n\nS = [input()...","H,W = map(int,input().split())\nS = []\nfor i ...",1
4,k = int(input())\ns = input()\n\nif len(s) <= ...,K = int(input())\nS = input()\nif len(S) <= K:...,1


In [9]:
# Code datasets (for labeled and unlabeled code snippets)

class CodePairDataset(Dataset):
    def __init__(self, codes_a, codes_b, labels, tokenizer):
        super().__init__()
        assert len(codes_a) == len(codes_b) == len(labels), "Length MUST match!"
        self.codes_a = codes_a
        self.codes_b = codes_b
        self.labels = labels
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        code_a = self.codes_a[idx]
        code_b = self.codes_b[idx]
        label = self.labels[idx]
        # Encode the sequences for sequence pair classification
        # ([CLS], code_a tokens , [SEP], code_b tokens, [SEP])
        encoding = self.tokenizer(
            code_a, code_b,
            padding='max_length',  # Pad to max_length
            max_length=self.tokenizer.model_max_length,
            truncation=True,       # Truncate to max_length
            return_tensors='pt'    # Return torch.Tensor objects
        )
        # Remove batch dimension
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}

        return encoding, label

    def __len__(self):
        return len(self.labels)

    @classmethod
    def from_csv_data(cls, path: str, tokenizer):
        df = pd.read_csv(path, header=0, names=COLUMNS)
        df = df.drop(columns=['pid', 'sid_1', 'sid_2'])

        # Filter out sequences that are longer then 4096 chararters
        MAX_CHAR_COUNT = 4096
        filter = lambda row: len(row['src_1']) + len(row['src_2']) < MAX_CHAR_COUNT
        df = df[df.apply(filter, axis=1)]
        print('filtered sequences:', df.shape)

        srcs_x = df['src_1'].to_list()
        srcs_y = df['src_2'].to_list()
        labels = df['label'].to_list()
        return cls(srcs_x, srcs_y, labels, tokenizer)

## Model

In [10]:
# Model for finetuning

class CodeSimilarityClassifier(nn.Module):
    def __init__(self,
        bert,  # BERT based model instance
        freeze_bert=False,
        dropout_rate=0.2,
    ):
        super().__init__()
        self.bert = bert
        self.drop = nn.Dropout(dropout_rate)
        self.cls = nn.Linear(self.bert.config.hidden_size, 1)
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, bert_input) -> torch.Tensor:
        with torch.device(device):
            bert_output = self.bert(**bert_input)
            pooler_output = bert_output.pooler_output
            pooler_output = self.drop(pooler_output)
            logits = self.cls(pooler_output)
            return logits

## Training

In [11]:
def train_one_epoch(
    model, device,
    loss_func,
    scaler,
    optimizer,
    scheduler,
    dataloader,
    iters_to_accumulate,
    # for logging
    print_every=10
):
    running_loss = 0.0
    model.train()

    num_iter = len(dataloader)
    for iter, (encoding, labels) in enumerate(tqdm(train_loader)):
        # Converting to cuda tensors
        for k, v in encoding.items(): encoding[k] = v.to(device)
        labels = labels.to(device)

        # Obtaining the logits from the model
        # Enables autocasting for the forward pass (model + loss)
        with autocast('cuda'):
            # Obtaining the logits from the model
            logits = model(encoding)
            # Computing loss
            loss = loss_func(logits.squeeze(-1), labels.float())
            # Normalize the loss because it is averaged
            loss = loss / iters_to_accumulate

        # Backpropagating the gradients
        # Scales loss. (calls backward() on scaled loss to create scaled gradients)
        scaler.scale(loss).backward()

        if (iter + 1) % iters_to_accumulate == 0:
            # Optimization step
            # scaler.step() first unscales the gradients of the optimizer's assigned params.
            # If these gradients do not contain infs or NaNs, opti.step() is then called,
            # otherwise, opti.step() is skipped.
            scaler.step(optimizer)
            # Updates the scale for next iteration.
            scaler.update()
            # Adjust the learning rate based on the number of iterations.
            scheduler.step()
            # Clear gradients
            optimizer.zero_grad()

        running_loss += loss.item()

        # Print training loss information
        if (iter + 1) % print_every == 0:
            print(
                f"Iteration {iter+1}/{num_iter} complete. " +
                f"Loss: {running_loss / print_every}"
            )
            running_loss = 0.0


def evaluate_loss(
    model, device,
    loss_func,
    dataloader
):
    model.eval()

    loss, count = 0,0
    with torch.no_grad():
        for it, (encoding, labels) in enumerate(tqdm(dataloader)):
            # Converting to cuda tensors
            for k, v in encoding.items(): encoding[k] = v.to(device)
            labels = labels.to(device)
            logits = model(encoding)
            loss += loss_func(logits.squeeze(-1), labels.float()).item()
            count += 1

    mean_loss = loss / count
    return mean_loss


def train_bert(
    model,
    loss_func,
    optimizer,
    scheduler,
    train_loader, valid_loader,
    epochs,
    iters_to_accumulate
):
    best_loss = np.Inf
    iters = []
    train_losses = []
    valid_losses = []

    scaler = GradScaler('cuda')

    for epoch in range(epochs):
        # Compute training loss
        train_loss = train_one_epoch(
            model, device, loss_func,
            scaler,  # grad scaler
            optimizer,
            scheduler,
            train_loader,
            iters_to_accumulate,
            # print the training loss 5 times per epoch
            print_every=len(train_loader) // 5
        )
        # Compute validation loss
        valid_loss = evaluate_loss(
            model, device, loss_func,
            valid_loader
        )
        print(f"Epoch {epoch+1} complete! Validation Loss: {valid_loss}")

        if valid_loss < best_loss:
            print(f"Best validation loss improved from {best_loss} to {valid_loss}")
            best_loss = valid_loss
            # TODO: save the model

    torch.cuda.empty_cache()

In [12]:
# Parameters

bert_name = "neulab/codebert-python"

"""
other checkpoints:
- "microsoft/codebert-base"
- "huggingface/CodeBERTa-small-v1"
"""

epochs = 4
lr = 1e-5  # learning rate
wd = 1e-5  # weight decay
bs = 20    # batch size
# The gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate.
# If set to "1", you get the usual batch size
iters_to_accumulate = 2

In [13]:
# Dataset

bert_tokenizer = AutoTokenizer.from_pretrained(bert_name)

dataset = CodePairDataset.from_csv_data(paired_dataset_url, bert_tokenizer)

train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_data, valid_data = random_split(dataset, [train_size, valid_size])

SHUFFLE = True
train_loader = DataLoader(train_data, batch_size=bs, shuffle=SHUFFLE)
valid_loader = DataLoader(valid_data, batch_size=bs, shuffle=SHUFFLE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


filtered sequences: (9881, 3)


In [14]:
# Model instance

bert_model = AutoModel.from_pretrained(bert_name).to(device)

model = CodeSimilarityClassifier(bert_model).to(device)

config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at neulab/codebert-python and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Optimizer and Scheduler

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

# The number of steps for the warmup phase.
num_warmup_steps = 0
# The total number of training steps
num_training_steps = epochs * len(train_loader)
# Necessary to take into account Gradient accumulation
num_training_steps = num_training_steps // iters_to_accumulate

scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [16]:
# Train the model

loss_func = nn.BCEWithLogitsLoss()

train_bert(
    model, loss_func,
    optimizer,
    scheduler,
    train_loader, valid_loader,
    epochs,
    iters_to_accumulate
)

  0%|          | 0/396 [00:00<?, ?it/s]

Iteration 79/396 complete. Loss: 0.3464324383796016
Iteration 158/396 complete. Loss: 0.33433768719057494
Iteration 237/396 complete. Loss: 0.33065754288359533
Iteration 316/396 complete. Loss: 0.3374228692507442
Iteration 395/396 complete. Loss: 0.3309842223608041


  0%|          | 0/99 [00:00<?, ?it/s]

Epoch 1 complete! Validation Loss: 0.6560228683731772
Best validation loss improved from inf to 0.6560228683731772


  0%|          | 0/396 [00:00<?, ?it/s]

...

Epoch 3 complete! Validation Loss: 0.6427471631705158


  0%|          | 0/396 [00:00<?, ?it/s]

Iteration 79/396 complete. Loss: 0.3046806001965004
Iteration 158/396 complete. Loss: 0.3088964582244052
Iteration 237/396 complete. Loss: 0.291930813766733
Iteration 316/396 complete. Loss: 0.3020920127252989
Iteration 395/396 complete. Loss: 0.30002661068228226


  0%|          | 0/99 [00:00<?, ?it/s]

Epoch 4 complete! Validation Loss: 0.6448821861936589
